In [608]:
#Task : Unzip the file
import zipfile
import os

# Unzip the file and storing into the 'data/' directory
os.makedirs('./data/', exist_ok=True)

with zipfile.ZipFile('hw0_dataset_1M.csv.zip', 'r') as zip_ref:
    zip_ref.extractall('./data/')




In [610]:
#Task: Loading the dataset into a data frame (df)
import pandas as pd

#load the CSV into a DataFrame
df = pd.read_csv( "./data/hw0_dataset_1M.csv", names = [i for i in range(0,30)], on_bad_lines="skip", low_memory=False)
# Displaying the first few rows to know the structure
print(df.head())
print("The above output is before cleaning the extra columns ")




#Ensuring that the second column (index 1) is treated as a string
df[1] = df[1].astype(str)
# Identifying rows where the second column (column with index 1) contains the word 'key'
header_rows = df[df[1].str.contains('key', case=False, na=False)]

# Finding the longest Header row
header_row = header_rows.loc[header_rows.notna().sum(axis=1).idxmax()]

# Setting this row as the new header
df.columns = header_row

# Remove extraneous rows (including the rows used as header)
df = df.drop(header_rows.index)

# Dropping columns where the header value is NaN
df_cleaned = df.loc[:, df.columns.notna()]

# Dropping any rows with all NaN values which is optional
df_cleaned = df_cleaned.dropna(how='all')

# Displaying the first few rows of the cleaned DataFrame
print(df_cleaned.head())

    0            1                2                        3           4   \
0  NaN          key  doi-asserted-by                      DOI  first-page   
1  0.0  e_1_2_7_2_1        publisher         10.1038/35041545         NaN   
2  1.0  e_1_2_7_3_1        publisher  10.1126/science.1155121         NaN   
3  2.0  e_1_2_7_4_1        publisher    10.1007/s003820050007         NaN   
4  3.0  e_1_2_7_5_1        publisher     10.1029/2000GL012471         NaN   

             5       6     7       8              9   ...  20  21  22  23  24  \
0  volume-title  author  year  volume  journal-title  ... NaN NaN NaN NaN NaN   
1           NaN     NaN   NaN     NaN            NaN  ... NaN NaN NaN NaN NaN   
2           NaN     NaN   NaN     NaN            NaN  ... NaN NaN NaN NaN NaN   
3           NaN     NaN   NaN     NaN            NaN  ... NaN NaN NaN NaN NaN   
4           NaN     NaN   NaN     NaN            NaN  ... NaN NaN NaN NaN NaN   

   25  26  27  28  29  
0 NaN NaN NaN NaN NaN  
1 

In [612]:
# Saving the cleaned data into 'data/cleaned_data_selected_columns.csv'
df_cleaned.to_csv('data/cleaned_data.csv', index=False)

# Confirmation message
print("Cleaned data has been saved to 'data/cleaned_data.csv'.")

Cleaned data has been saved to 'data/cleaned_data.csv'.


In [614]:
# Task: Extract,transform and export JSON data
import pandas as pd
import re

# Restrict the data to the required 3 columns: 'DOI', 'journal-title', and 'doi-asserted-by'
df_transformed = df_cleaned[['DOI', 'journal-title', 'doi-asserted-by']].copy()

# Normalize the DOI column (convert to lowercase) using .loc[] 
df_transformed['DOI'] = df_transformed['DOI'].str.lower()


df_transformed['journal-title'] = df_transformed['journal-title'].apply(lambda x: re.sub(r'\s+', ' ', x.strip()) if pd.notnull(x) else x)

# Removing all punctuation from 'journal-title'
df_transformed['journal-title'] = df_transformed['journal-title'].apply(lambda x: re.sub(r'[^\w\s]', '', x) if pd.notnull(x) else x)

import os
os.makedirs('data', exist_ok=True)

# Save the transformed data into a CSV file
csv_filename = 'data/data_cleaned_normalized.csv'
df_transformed.to_csv(csv_filename, index=False)

# Save the transformed data into a JSON file
json_filename = 'data/data_cleaned_normalized.json'
df_transformed.to_json(json_filename, orient='records', lines=True)

# Confirmation message
print(f"Data has been saved to '{csv_filename}' and '{json_filename}'.")

Data has been saved to 'data/data_cleaned_normalized.csv' and 'data/data_cleaned_normalized.json'.


In [616]:
#Task: Filter, transform and export CSV data.
import pandas as pd
import os

# Function to filter data for rows with DOI and journal-title, ensure unique DOIs, and export only those columns
def filter_and_export_data(df, csv_output_path, json_output_path):
    # Filter rows where both DOI and journal-title are not empty
    df_filtered = df.dropna(subset=['DOI', 'journal-title'])

    # Remove duplicate DOIs (keeping only the first occurrence)
    df_filtered_unique = df_filtered.drop_duplicates(subset='DOI')

    # Debugging step: Print the current columns to verify selection
    print("Columns before selection:", df_filtered_unique.columns)

    # Select only DOI and journal-title columns
    df_filtered_final = df_filtered_unique[['DOI', 'journal-title']].copy()  # Select only these two columns

    # Debugging step: Verify that the resulting DataFrame only contains the two columns
    print("Columns after selection:", df_filtered_final.columns)

    # Ensure the 'data' directory exists
    os.makedirs(os.path.dirname(csv_output_path), exist_ok=True)

    # Export to CSV
    df_filtered_final.to_csv(csv_output_path, index=False)

    # Export to JSON
    df_filtered_final.to_json(json_output_path, orient='records', lines=True)

    print(f"Filtered data saved to '{csv_output_path}' and '{json_output_path}'.")

# Example usage assuming df_cleaned is available
csv_output_path = 'data/data_filtered.csv'
json_output_path = 'data/data_filtered.json'

# Call the function to filter, transform, and export data
filter_and_export_data(df_cleaned, csv_output_path, json_output_path)


Columns before selection: Index(['key', 'doi-asserted-by', 'first-page', 'DOI', 'article-title',
       'volume', 'author', 'year', 'journal-title', 'unstructured', 'issue',
       'series-title', 'volume-title', 'edition', 'ISSN', 'issn-type',
       'isbn-type', 'ISBN'],
      dtype='object', name=42114)
Columns after selection: Index(['DOI', 'journal-title'], dtype='object', name=42114)
Filtered data saved to 'data/data_filtered.csv' and 'data/data_filtered.json'.


In [618]:
#Task: Write functions to get journal information.

def n_most_frequent_doi(df, n):
    # Get the frequency count of DOIs and sort them in descending order
    doi_counts = df['DOI'].value_counts().head(n)
    
    # Return the DataFrame containing the n most frequent DOIs
    return doi_counts.reset_index().rename(columns={'index': 'DOI', 'DOI': 'count'})

def journal_lookup(df, s):
    # Filter rows where the journal-title contains the substring s (case insensitive)
    filtered_df = df[df['journal-title'].str.contains(s, case=False, na=False)]
    
    # Return the filtered DataFrame
    return filtered_df

# Assuming df_cleaned is your cleaned DataFrame

# Get the 5 most frequent DOIs
most_frequent_dois = n_most_frequent_doi(df_cleaned, 5)
print(most_frequent_dois)

# Lookup journals with 'Science' in the title
journal_results = journal_lookup(df_cleaned, 'Science')
print(journal_results)


       count  count
0  publisher  21037
1   crossref  14421
2          1   1096
3     2006.0    828
4     2007.0    791
42114                                   key doi-asserted-by  \
22        c2cs35095e-(cit3)/*[position()=1]        crossref   
59       c2cs35095e-(cit44)/*[position()=1]        crossref   
86       c2cs35095e-(cit74)/*[position()=1]        crossref   
89       c2cs35095e-(cit77)/*[position()=1]        crossref   
108      c2cs35095e-(cit96)/*[position()=1]        crossref   
...                                     ...             ...   
992036    10.1016/j.jastp.2009.03.026_bib43        crossref   
992053                       erlac36f6bib16       publisher   
992222   10.1016/S1352-2310(00)00275-2_BIB1        crossref   
992226   10.1016/S1352-2310(00)00275-2_BIB5        crossref   
992249  10.1016/S1352-2310(00)00275-2_BIB29        crossref   

42114                                        first-page     DOI  \
22                              10.1126/science.1210026 